In [73]:
import torch.nn as nn

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 4060 Laptop GPU


In [72]:
class AlphaDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
class AlphaNet(nn.Module):
    # Neural Network for Alpha Prediction
    def __init__(self, input_dim, hidden_dims = [128, 64, 32],
                 dropout_rate = 0.3, use_batch_norm = True):
        super().__init__()

        layers = []
        prev_dim = input_dim

        for i, hidden_dim in enumerate(hidden_dims):
            # Linear Layer
            layers.append(nn.Linear(prev_dim, hidden_dim))

            # Batch normalization
            if use_batch_norm:
                layers.append(nn.BatchNorm1d(hidden_dim))

            # Activation
            layers.append(nn.ReLU())

            # Drop out
            if i < len(hidden_dim) -1:
                layers.append(nn.Dropout(dropout_rate))

            prev_dim = hidden_dim

        # Output Layer
        layers.append(nn.Linear(prev_dim, 1))

        self.network = nn.Sequential(*layers)

        # Initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            nn.init.constant_(module.bias, 0)
    
    def forward(self, x):
        return self.network(x).squeeze()

In [ ]:
class AlphaTrainer:
    def __init__(self, model_config):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # Default
        self.config = {
            'hidden_dims': [128, 64, 32],
            'dropout_rate': 0.3,
            'use_batch_norm': True,
            'learning_rate': 0.001,
            'batch_size': 512,
            'epochs': 100,
            'patience': 15,
            'weight_decay': 1e-5,
            'scheduler_patience': 7,
            'scheduler_factor': 0.5
        }
        if model_config:
            self.config.update(model_config)
        
        # Feature names
        self.feature_names = [
            'daily_return', 'FEAT_DebtEquity_quarterly', 'FEAT_PE_quarterly',
            'FEAT_EVEBITDA_quarterly', 'FEAT_ROE_quarterly', 'price_mom_5d',
            'price_mom_10d', 'price_mom_20d', 'vol_w_mom_5d', 'vol_w_mom_10d',
            'vol_w_mom_20d', 'volatility_10d', 'skewness_20d', 'rsi_14d',
            'zscore_mom_10d_60w'
        ]

        # Scalers
        self.feature_scaler = RobustScaler()
        self.target_scaler = StandardScaler()

        # Training history
        self.history = {'train_loss': [], 'val_loss': [], 'ic': [], 'rank_ic': []} 
    
    def prepare_data(self, stock_df_path, market_df_path, test_size = 0.2):
        features_cols = self.feature_names
        dfs = []
        for path in stock_df_path:
            p = Path(path).resolve()
            exchange = p.parts[-2]
            df = pd.read_csv(p)
            df['exchange'] = exchange
            dfs.append(df)
        df_stock_all = pd.concat(dfs, ignore_index= True)
        dfs = []
        for path in market_df_path:
            p = Path(path).resolve()
            name = p.stem.upper()
            exchange = "HNX" if "HNX" in name else "HOSE"
            df = pd.read_csv(p)
            df['exchange'] = exchange
            dfs.append(df)
        df_market_all = pd.concat(dfs, ignore_index= True)
        df_market_subset = df_market_all[['time', 'exchange', 'FUT_RET_10D_market']]        
        merged_df = pd.merge(df_stock_all, df_market_subset, on = ['time', 'exchange'], how = 'inner' )
        merged_df['alpha'] = merged_df['FUT_RET_10D'] - merged_df['FUT_RET_10D_market']
        
        #Separate features and target
        X = merged_df[features_cols].values
        y = merged_df['alpha'].values

        #Train test split
        split_idx = int(len(X) * (1 - test_size))
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]

        #Scale features and target
        X_train_scaled = self.feature_scaler.fit_transform(X_train)
        X_test_scaled = self.feature_scaler.transform(X_test)
        y_train_scaled = self.target_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
        y_test_scaled = self.target_scaler.transform(y_test.reshape(-1, 1)).flatten()

        print(f"Training set: {X_train_scaled.shape}, Test set: {X_test_scaled.shape}")

        return (X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled,
                y_train, y_test, features_cols)
    
    def calculate_ic(self, predictions, targets):
        # Remove NaN values
        valid_mask = ~(np.isnan(predictions) | np.isnan(targets))
        pred_clean = predictions[valid_mask]
        target_clean = targets[valid_mask]
        
        if len(pred_clean) < 10:  # Need minimum samples
            return 0.0, 0.0
        
        # Pearson correlation (IC)
        ic = np.corrcoef(pred_clean, target_clean)[0, 1]
        if np.isnan(ic):
            ic = 0.0
        
        # Spearman rank correlation (Rank IC)
        from scipy.stats import spearmanr
        rank_ic, _ = spearmanr(pred_clean, target_clean)
        if np.isnan(rank_ic):
            rank_ic = 0.0
        
        return ic, rank_ic
    
    def train_model(self, X_train, X_val, y_train, y_val, y_val_unscaled):
        # Train the neural network
        
        # Create datasets and dataloaders
        train_dataset = AlphaDataset(X_train, y_train)
        val_dataset = AlphaDataset(X_val, y_val)
        train_loader = DataLoader(train_dataset, batch_size= self.config['batch_size'], shuffle = True)
        val_loader = DataLoader(val_dataset, batch_size = self.config['batch_size'], shuffle= False)

        # Initialize mode
        model = 

In [ ]:
def train_alpha_model(stock_df_path, market_df_path, config):

    trainer = AlphaTrainer(config)

    # Prepare data
    X_train, X_test, y_train_scaled, y_test_scaled, y_train, y_test, features_cols = trainer.prepare_data(stock_df_path, market_df_path)

    # Train model
    #model = trainer.train_model(X_train, X_test, y_train_scaled, y_test_scaled, y_test)

    

In [64]:
import os
import glob
current_script_directory = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
input_base_folder = os.path.join(current_script_directory, '..', 'data_prep')
stock_features_path = os.path.join(input_base_folder, 'calculated_stock_features')
market_indices_path = os.path.join(input_base_folder, 'stock_data\market_indices')

In [65]:
stock_df_path = glob.glob(os.path.join(stock_features_path, "*", "*.csv"))
market_df_path = glob.glob(os.path.join(market_indices_path, "*.csv"))

In [66]:
print(market_df_path)

['c:\\Users\\ADMIN\\PROJECT\\alpha_signal_predictor\\models\\..\\data_prep\\stock_data\\market_indices\\HNXINDEX.csv', 'c:\\Users\\ADMIN\\PROJECT\\alpha_signal_predictor\\models\\..\\data_prep\\stock_data\\market_indices\\VNINDEX.csv']


In [67]:
train_alpha_model(stock_df_path, market_df_path, 0)

Training set: (666792, 15), Test set: (166698, 15)
Training set: (666792,), Test set: (166698,)
